In [1]:
#Running on Colab
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=3929933b3a17babbbbc2750a44f3ecbc218cc0347965def37072fb39560be322
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [3]:
#  Importing Required Libraries
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

# Create Spark session and ContextRun PySpark.
# create the session
conf = SparkConf().set("spark.ui.port","4050")
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.appName("DataFrame").config('spark.ui.port', '4050').getOrCreate()
spark

**Adjacency List to Edge List Conversion**

In [19]:
# Read the file and create the RDD
rdd1 = sc.textFile('file.txt').map(lambda line: (line.split('->')[0], line.split('->')[1].split(',')))
Rdd = rdd1.flatMap(lambda x: [(x[0],y) for y in x[1]])

# Collect the results
Rdd.collect()

[('1', '2'),
 ('1', '4'),
 ('2', '3'),
 ('2', '4'),
 ('3', '1'),
 ('4', '0'),
 ('5', '6'),
 ('5', '2'),
 ('6', '3'),
 ('7', '2')]

**Edge List to Adjacency List Conversion**

In [17]:
# Read the file and create the RDD
rdd1 = sc.textFile('file2.txt').map(lambda line: (line.split(',')[0], line.split(',')[1]))
rdd2=rdd1.groupByKey().collect()

for vertex,neigbours in rdd2:
  print('\n',vertex,'-> ',end='')
  for neigbour in neigbours:
    print(neigbour,end=' ')


 1 -> 2 4 
 4 -> 0 
 2 -> 3 4 
 3 -> 1 
 5 -> 6 2 
 6 -> 3 
 7 -> 2 

**Page Rank Simple Algorithm** (Not sure if correct)

In [44]:
rdd=sc.textFile('file3.txt').map(lambda line: (line.split('->')[0],line.split('->')[1].split(',')))
#  First Type of Packet Generation (Neighbours of Node)
rddP1=rdd.map(lambda point: (point[0].split(',')[0],point[1]))
print('First Type of Packet Generation (Neighbours of Node)')
print(rddP1.collect())
#  Second Type of Packet Generation (Importance of Node)
rddP2=rdd.flatMap(lambda point: [(p,float(point[0].split(',')[1])/len(point[1])) for p in point[1]])
print('Second Type of Packet Generation (Importance of Node)')
print(rddP2.collect())
rddP2Sum=rddP2.reduceByKey(lambda x,y : x+y)
print('Total Importance of Each Node')
rddP2Sum.collect()

First Type of Packet Generation (Neighbours of Node)
[('1', ['2', '4']), ('2', ['3', '4']), ('3', ['1']), ('4', ['0']), ('5', ['6', '2']), ('6', ['3']), ('7', ['2'])]
Second Type of Packet Generation (Importance of Node)
[('2', 0.4), ('4', 0.4), ('3', 0.4), ('4', 0.4), ('1', 0.8), ('0', 0.8), ('6', 0.4), ('2', 0.4), ('3', 0.8), ('2', 0.8)]
Total Importance of Each Node


[('4', 0.8),
 ('1', 0.8),
 ('0', 0.8),
 ('2', 1.6),
 ('3', 1.2000000000000002),
 ('6', 0.4)]

In [45]:
# Chatgpt code
# Read data from file and create the RDD
rdd = sc.textFile('file3.txt').map(lambda line: (line.split('->')[0], line.split('->')[1].split(',')))

# Initial PageRank values for each node
initial_rank = 1.0

# Damping factor for PageRank calculation
damping_factor = 0.85

# Number of iterations for PageRank computation
iterations = 10

# Function to compute contributions of neighbors
def compute_contributions(neighbors, rank):
    num_neighbors = len(neighbors)
    for neighbor in neighbors:
        yield (neighbor, rank / num_neighbors)

# Iterative PageRank computation
for i in range(iterations):
    # First type of packet generation: Neighbors of each node
    rdd_neighbors = rdd.map(lambda point: (point[0].split(',')[0], point[1]))
    print('First Type of Packet Generation (Neighbors of Node)')
    print(rdd_neighbors.collect())

    # Second type of packet generation: Importance of each node
    rdd_importance = rdd.flatMap(lambda point: compute_contributions(point[1], float(point[0].split(',')[1])))
    print('Second Type of Packet Generation (Importance of Node)')
    print(rdd_importance.collect())

    # Summing up the importance of each node
    rdd_sum = rdd_importance.reduceByKey(lambda x, y: x + y)
    print('Total Importance of Each Node after Iteration', i+1)
    print(rdd_sum.collect())

    # Update PageRank values for the next iteration
    rdd = rdd_neighbors.join(rdd_sum).mapValues(lambda x: (x[0], damping_factor * x[1] + (1 - damping_factor)))

# Collect final PageRank values
final_ranks = rdd.collect()
print('Final PageRank Values:')
print(final_ranks)


First Type of Packet Generation (Neighbors of Node)
[('1', ['2', '4']), ('2', ['3', '4']), ('3', ['1']), ('4', ['0']), ('5', ['6', '2']), ('6', ['3']), ('7', ['2'])]
Second Type of Packet Generation (Importance of Node)
[('2', 0.4), ('4', 0.4), ('3', 0.4), ('4', 0.4), ('1', 0.8), ('0', 0.8), ('6', 0.4), ('2', 0.4), ('3', 0.8), ('2', 0.8)]
Total Importance of Each Node after Iteration 1
[('4', 0.8), ('1', 0.8), ('0', 0.8), ('2', 1.6), ('3', 1.2000000000000002), ('6', 0.4)]
First Type of Packet Generation (Neighbors of Node)
[('4', (['0'], 0.8300000000000001)), ('3', (['1'], 1.17)), ('6', (['3'], 0.49000000000000005)), ('1', (['2', '4'], 0.8300000000000001)), ('2', (['3', '4'], 1.5100000000000002))]
Second Type of Packet Generation (Importance of Node)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 69.0 failed 1 times, most recent failure: Lost task 1.0 in stage 69.0 (TID 137) (a6219f99ed85 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwa